## Analisis con PySpark.

### 1) subir los csv en databricks con una notebook "origin-to-raw" en una carpeta que diga "shop-carrito" seguido de una carpeta 'raw-data', ejemplo {contenedor}/shop-carrito/raw-data/shop_carrito.csv 

### 2) En un nuevo notebook "raw-to-trusted", realizar los joins correspondientes de las 7 tablas, la forma de unirlos es la siguiente:

• Shop-carrito -> shop-clientes por el campo “id_cliente”

• Shop-carrito -> shop_campanias por el campo “id_campania”

• Shop-carrito -> shop_articulos por el campo “id_articulo_ppal”

• Shop-carrito -> shop_formapago por el campo “id_fpago”

• Shop-carrito -> shop_carrito_estados por el campo “estado”

• Shop-carrito -> shop_carrito_estados_pago por el campo “estado_pago”

Al finalizar guardar el tablon en formato parquet en una carpeta que diga Trusted, ejemplo {contenedor}/trusted/tablon_shop_carrito/

### 3) una vez que tengas el tablon shop_carrito, realizar los analisis correspondientes:

a) Quiero saber qué cantidad de solicitudes de compra se encuentran en cada estado, y el porcentaje en relación al total.

b) Quiero conocer los clientes principales de la empresa, generando un ranking por cantidad y otro por monto, teniendo en cuenta solamente las compras finalizadas (efectuadas).

c) Queremos ver cuáles fueron los medios de pago más utilizados por los clientes

d) Realizar un analisis donde me determine las campanias mas eficaces.

e) realizar 2 analisis mas y que nos expliques los resultados obtenidos.

# origin-to-raw

In [0]:
datalake = 'dlformacion'
contenedor_origen = 'shop-carrito'
contenedor_destino = 'alejo-menini'
tablas = ['articulos', 'campanias_gp', 'shop_carrito_estados_pago', 'shop_carrito_estados', 'Shop_carrito','Shop_clientes','shop_formapago']

In [0]:
#access_key = dbutils.secrets.get(scope = "dlformacion", key = "key1")
access_key = 'FIPeodgc8WLFK0TvcI+UHily5K2uvBp5AeryqRvqsc6fEapQj8vfEOhZs5R8xl9YX0MhvIaXK8L5+AStPinFvw=='
spark.conf.set(f'fs.azure.account.key.{datalake}.dfs.core.windows.net', access_key)

Importamos los csv de nuestro datalake

In [0]:
for tabla in tablas:
    separador = ',' if (tabla in ['articulos', 'campanias_gp']) else ';'
    exec(f'''df_{tabla} = spark.read.load(
        f"abfss://{contenedor_origen}@{datalake}.dfs.core.windows.net/{tabla}.csv",
        format = "csv",
        sep = "{separador}",
        header = "true")''')

In [0]:
df_Shop_carrito.limit(5).display()

id,fecha,fecha_unix,id_cliente,direccion,numero,piso,dpto,entrecalles,cp,localidad,provincia,telefono,estado,estado_fecha,estado_pago,corte_generado_fecha,corte_generado,courier,id_fpago,comentarios,comentarios_internos,id_operador,tarjeta_numero,tarjeta_vencimiento,tarjeta_codigo,tarjeta_titular,tarjeta_pagos,tarjeta_aprobacion,fecha_dia,fecha_mes,fecha_ano,fecha_hora,campania,id_campania,monto,nro_guia,fecha_facturado,id_lote,id_estado_lote,ano,t_entrega,nota_credito,estado_logistico,id_oca,id_articulo_ppal,status_tango,fecha_anulado,fecha_pago_pf,fecha_aprobacion_bo,email,telefono_alternativo,id_pais_negocio,cotizacion,mail_fc_env,inbound,monto_DLS,fecha_entrega,franja_horaria,sms_enviado,fecha_entregado,asiento_refacturacion,id_courier,epack,id_subarea,costo_total,export_hermes,hermes,export_bi,mail_ap_bo,indice_hermes,monto_neto_usd,wp_notific,donacion,fecha_fin_tratamiento,pae_final,id_campania_recalc,costo_logistica,recompra_empresa
7,7/3/2016 08:37,1457393857,27,Pasaje trufan,579,null,null,null,637,PUENTE ALTO,PUENTE ALTO,null,30,null,3,null,0,MotoPartner,2,null,null,1,1234,1212,123,Eliseo Del Carmen Guevara Mu�oz,6,123,8,3,2016,02:37,null,32,7728,122,1459957162,33,1,null,12-24 Hs,0,1,null,0,0,1461812400,0,1457394565,null,991231234,0,0.0,0,0,0.0,7/3/2016,16 a 20,0,0,0,2,null,0,0.0,0,0,1,0,0,0.0,0,0,0,0,0,11.4,0
62,6/3/2016 02:14,1457284452,25,Esmeralda,5756,null,null,Prueba Entre calles,643,SAN MIGUEL,SAN MIGUEL,null,5,null,2,null,0,MotoPartner,3,Comentarios corrier,comentarios internos,1,1235,null,null,null,6,1111,6,3,2016,20:14,null,22,81244,null,0,0,0,null,12-24 Hs,0,null,null,197,0,0,0,1457284555,null,966252103,3,683.89,0,0,118.8,6/3/2016,16 a 20,0,0,0,2,null,0,0.0,0,0,0,0,0,99.83,0,0,0,0,0,11.4,0
63,7/3/2016 12:09,1457363388,25,Esmeralda,5756,null,null,Prueba Entre calles,643,SAN MIGUEL,SAN MIGUEL,null,30,null,1,null,1,MotoPartner,1,null,null,6,null,null,null,null,null,null,7,3,2016,18:09,null,22,47980,1,1457408615,3,3,null,12-24 Hs,0,null,null,198,1,0,0,1457363440,null,966252103,3,683.89,0,0,70.16,7/3/2016,10 a 14,0,0,0,2,null,0,0.0,0,0,1,0,0,58.96,0,0,0,0,0,11.4,0
64,7/3/2016 05:18,1457381895,26,PSJE TRUPAN,579,null,null,COBQUECURA Y QUIRIHUE,637,PUENTE ALTO,PUENTE ALTO,null,30,null,3,null,1,MotoPartner,1,null,PARA DESPACHO 1ERO ABRIL POR INVENTARIO,15,null,null,null,null,null,null,7,3,2016,23:18,null,32,171970,83,1459860715,28,3,null,12-24 Hs,0,null,null,198,1,1461812400,0,1459821479,null,998437788,3,683.89,0,0,251.46,7/3/2016,10 a 14,0,0,0,2,null,5,0.0,0,0,1,0,0,211.31,0,0,0,0,0,11.4,0
65,7/3/2016 06:09,1457384975,27,PEDRO RIVEROS,1583,null,314,TORRE ARAUCARIA,638,QUILICURA,QUILICURA,null,30,null,3,null,0,Tarjeta MotoPartner,6,Sector Valle lo Campino,Venta Chat Green Pharma Pelofort,2,null,null,null,null,1,null,8,3,2016,00:09,null,48,88980,131,1459958029,32,3,null,12-24 Hs,0,null,null,198,0,0,0,1459821561,null,984184924,3,683.89,0,0,130.11,7/3/2016,10 a 14,0,0,0,5,null,5,0.0,0,0,1,0,0,109.34,0,0,0,0,0,11.4,0


Y los guardamos en nuestro contenedor, en formato csv en shopr-carrito/raw_data

In [0]:
for tabla in tablas:
    exec(f"""df_{tabla}.write.csv(
        path=f'abfss://{contenedor_destino}@{datalake}.dfs.core.windows.net/shop-carrito/raw_data/{tabla}.csv',
        header=True,
        mode = 'overwrite')""")